In [ ]:
# Copyright 2025 Sony Corporation

# make table4 

In [ ]:
import subprocess
import os
import glob
import shutil
import sys
import pandas as pd
from multiprocessing import Process

In [ ]:
workdir = os.getcwd()

In [ ]:
#fixed parameter 
meta = 38
iteration = 10
seed = 1
max_process=10 #10 para
orders = 10

#clust
div_num = 100
fold_num = 10

#meta
div_num = meta
fold_num = 10

## make reordered fcs

In [ ]:
cmd = "RScript.exe R/make_reordered_samusik.R ./benchmark_data_sets/Samusik_all/data/Samusik_all.fcs ./benchmark_data_sets/Samusik_reorder"
proc = subprocess.Popen(cmd.split())
proc.wait()

# make table for reoeder

In [ ]:
cmd = "RScript.exe R/make_samusik_transtable.R ./benchmark_data_sets/Samusik_reorder ./benchmark_data_sets/Samusik_reorder"
proc = subprocess.Popen(cmd.split())
proc.wait()

## set sample for clustering

In [ ]:
Samplefcs_list = []
for i in range(1,11,1):
    fcsfile = "samsik_all_head%d.fcs"%i
    fcs = os.path.join(workdir,"benchmark_data_sets","Samusik_reorder",fcsfile)
    Samplefcs_list.append(fcs)

In [ ]:
FSOM_no_hang_seed_ev = "evaluate_FlowSOM_hang.R"

In [ ]:
experiment_name = "Samusik_jaccard-fsom"
os.chdir(workdir)
os.makedirs(experiment_name,exist_ok=True)

proc_list = []
total_num = 0
loop=10
for Samplefcs in Samplefcs_list:
    sample_name = os.path.splitext(os.path.basename(Samplefcs))[0]
    exp_path = os.path.join(workdir,experiment_name,sample_name)
    os.makedirs(exp_path,exist_ok=True)
    cmd = "RScript.exe cytometry-clustering-comparison/run_methods/run_FlowSOM2_variable_iter_seed_meta_sample_jaccard.R %s %s %d %d %d %s"%(Samplefcs,exp_path, iteration, seed, meta,"Samusik_all")
    proc = subprocess.Popen(cmd.split())
    proc_list.append(proc)
    total_num = total_num +1
    if (total_num) % max_process == 0 or (total_num == loop):
        #wait process
        for subproc in proc_list:
            subproc.wait()
        proc_list = []    

## evaluation

In [ ]:
import subprocess
os.chdir(workdir)
os.chdir("cytometry-clustering-comparison/evaluate_results")

proc_list = []
total_num = 0
cwd = os.getcwd()
for Samplefcs in Samplefcs_list:
    sample_name = os.path.splitext(os.path.basename(Samplefcs))[0]
    exp_path = os.path.join(workdir,experiment_name,sample_name) 
    cmd = "RScript.exe %s %s %s/FlowSOM_labels_seed_%d_iter_%d_meta_%d.txt %s/vseed_f1hang_%03d_iter_10_meta_%d.resh"%(FSOM_no_hang_seed_ev,Samplefcs,exp_path,seed,iteration,meta,exp_path,seed,meta)
    proc = subprocess.Popen(cmd.split())
    proc_list.append(proc)
    total_num = total_num +1
    if (total_num) % max_process == 0 or (total_num == loop):
        #wait for process
        for subproc in proc_list:
            subproc.wait()
        proc_list = []

In order to calcurate the jaccard index, the clustering resutls have to be restored to their original order.
We should change clustering results to original order.
Frome here, we change the clustering order.

In [ ]:
from workers import save_trans_result,calc_jaccard_meta_table4,calc_jaccard_clust_table4

In [ ]:
os.chdir(workdir)
import shutil

ref_df = pd.read_csv("benchmark_data_sets/Samusik_reorder/samusik_table_reference.csv")
proc_list = []
#clust
for i in range(2,iteration+1):
    clust_fname = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_clust.txt"%(experiment_name,i)
    table_fname = "benchmark_data_sets/Samusik_reorder/samusik_table_fold%d.csv"%i
    out_fname   = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_clust_trans.txt"%(experiment_name,i)
    
    proc = Process(target=save_trans_result,args=(ref_df, clust_fname, table_fname, out_fname))
    proc.start()
    proc_list.append(proc)
    
for p in proc_list:
    p.join()

i=1
#copy head1 file
clust_fname = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_clust.txt"%(experiment_name,i)
out_fname   = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_clust_trans.txt"%(experiment_name,i)
shutil.copyfile(clust_fname, out_fname)

In [ ]:
proc_list = []

#meta
for i in range(2,iteration+1):
    clust_fname = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_meta_38.txt"%(experiment_name,i)
    table_fname = "benchmark_data_sets/Samusik_reorder/samusik_table_fold%d.csv"%i
    out_fname   = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_meta38_trans.txt"%(experiment_name,i)
    
    proc = Process(target=save_trans_result,args=(ref_df, clust_fname, table_fname, out_fname))
    proc.start()
    proc_list.append(proc)
    
for p in proc_list:
    p.join()

i=1
#copy head1 file
clust_fname = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_meta_38.txt"%(experiment_name,i)
out_fname   = "%s/samsik_all_head%d/FlowSOM_labels_seed_1_iter_10_meta38_trans.txt"%(experiment_name,i)
shutil.copyfile(clust_fname, out_fname)    

In [ ]:
def df2jacdf(indf):
    max = indf[0].max() 
    index_list = []   
    for i in range(1,max + 1):
        ind = indf.loc[indf[0]==i].index 
        ind = ind+1 
        index_list.append(ind.tolist())
    df = pd.DataFrame(index_list)
    df.index = df.index + 1 
    return df

In [ ]:
# random init 
Outdir = "Samusik-jaccard-reorder"
base_dir = os.path.join(workdir,experiment_name)

#clust
for order in range(1,orders+1):
    sample_name = "samsik_all_head%d"%order
    out_dir = os.path.join(workdir, Outdir)
    os.makedirs(out_dir,exist_ok=True)

    in_file = os.path.join(base_dir,"samsik_all_head%d"%order, "FlowSOM_labels_seed_1_iter_10_clust_trans.txt")
    out_filename = "fsom_%s_jaccard_clust.csv"%(sample_name)
    out_path = os.path.join(out_dir, out_filename)
    #read file
    df = pd.read_csv(in_file, header=None,skiprows=1)
    out_df = df2jacdf(df) # make jaccard file 
    a = out_df.T
    a.to_csv(out_path, index=False, header=True)
    print(out_path)

In [ ]:
#meta

for order in range(1,seed_num + 1):
    sample_name = "samsik_all_head%d"%order
    out_dir = os.path.join(workdir, Outdir,"FSOM",sample_name)

    in_file =  os.path.join(base_dir,"samsik_all_head%d"%order, "FlowSOM_labels_seed_1_iter_10_meta38_trans.txt")
    out_filename = "fsom_%s_jaccard_meta.csv"%(sample_name)
    out_path = os.path.join(Outdir, out_filename)

    #read file
    df = pd.read_csv(in_file, header=None,skiprows=1)
    out_df = df2jacdf(df) # make jaccard file
    a = out_df.T
    a.to_csv(out_path, index=False, header=True)
    print(out_path)
print("done")    

In [ ]:
proc_list = []
Outdir = "Samusik-jaccard-reorder"

div_num = 100
fold_num = 10
proc = Process(target=calc_jaccard_clust_table4,args=(Outdir,workdir,fold_num, div_num))
proc.start()
proc_list.append(proc)

div_num = meta
fold_num = 10
proc = Process(target=calc_jaccard_meta_table4,args=(Outdir,workdir,fold_num, div_num))
proc.start()
proc_list.append(proc)

for p in proc_list:
    p.join()

In [ ]:
clust = pd.read_csv("samsik_reorder_jaccard_fsom_clust_table4.csv")

In [ ]:
clust["4"].describe()

In [ ]:
meta_df = pd.read_csv("samsik_reorder_jaccard_fsom_meta_table4.csv")

In [ ]:
meta_df["4"].describe()